Created on 4/2/2020
Author: Yuan-Chi Yang

Objective: template for content analysis on political feedback, please feel free to modify and play around

In [3]:
import pandas as pd
import nltk
import re

# Importing the data and perform some checks

The data is from "C:\Users\yyang60\PostDoct-Emory\medicaid-project\medicaid-classifier\labeling-data\whole_dataset\BERT_20200228\political-tweets-streaming.csv"

It consists of all the tweets classified as the 'p' class by the best performing classfiers to date.

In [4]:
df = pd.read_csv('./political-tweets-streaming.csv',header = 0, keep_default_na=False,dtype={'tweet_id':str})

In [5]:
len(df)

383969

In [6]:
df.columns

Index(['tweet_id', 'unprocessed_text', 'class'], dtype='object')

### Check Duplicates
No Duplicated

In [7]:
df.duplicated(subset = ['tweet_id'], keep=False).sum()

0

### Include 'text_remove_stopwords' column

In [11]:
def loadStopWords(FILENAME):
    stopword_list = []
    infile = open(FILENAME)
    for line in infile:
        stopword_list.append(line.strip())
    print(len(stopword_list))
    return stopword_list

In [12]:
def processing_text_remove_stopwords(tweet_text,stop_words):
    tweet_text = re.sub(r'&amp;', "and", tweet_text)
    tweet_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',
                        tweet_text)
    #tweet_text = re.sub('(@[A-Za-z0-9\_]+)', '', tweet_text)
    tweet_text = re.sub('(@[\S]+)', '', tweet_text)#added Angel
    ####the following section separate words following each hashtag
    list_hashtag1=re.findall('(#[\S]+)',tweet_text)
    list_hashtag2=[]
    for p in list_hashtag1:
        p=re.sub('(#)','',p)#string
        p1=re.findall('([A-Z]{2,})',p)#list
        for p2 in p1:
            list_hashtag2.append(p2)
        p=re.sub('([A-Z]{2,})',' ',p)
        p1=re.findall('([A-Z][a-z]{1,})',p)
        for p2 in p1:
            list_hashtag2.append(p2)
        p=re.sub('([A-Z][a-z]{1,})',' ',p)
        p=re.sub('([A-Z])',' ',p)
        p=re.sub('([^A-Za-z])',' ',p)
        p1=p.split()
        for p2 in p1:
            list_hashtag2.append(p2)
    tweet_text=re.sub('(#[\S]+)','',tweet_text)
    
    tweet_text = re.sub("[^a-zA-Z_-]", " ", tweet_text)
    tweet_text = tweet_text.lower()
    tweet_text = re.sub(r'\s{2,}', " ", tweet_text)
    list_hashtag= [h for h in list_hashtag2 if (not h in stop_words and len(h)>3)]
    tweet_text = [t for t in tweet_text.split() if (not t in stop_words and len(t)>3)]
    tweet_text.extend(list_hashtag)
    return ' '.join(tweet_text)

In [15]:
stopwords = set(loadStopWords('./stopwords.txt'))

261


In [16]:
df['text_remove_stopwords'] = df['unprocessed_text'].apply(lambda x: processing_text_remove_stopwords(x,stopwords))

## Find the most frequent words in text

In [17]:
unigrams = ' '.join(df['text_remove_stopwords'].to_list()).split()
bigrams = []
for tweet in df['text_remove_stopwords'].to_list():
    bigrams += list(nltk.bigrams(tweet.split()))
    
trigrams = []
for tweet in df['text_remove_stopwords'].to_list():
    trigrams += list(nltk.trigrams(tweet.split()))
uni_fd = nltk.FreqDist(unigrams)
big_fd = nltk.FreqDist(bigrams)
trig_fd = nltk.FreqDist(trigrams)


In [19]:
num = 30

print('unigrams:\n')
fd = uni_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
for i in range(0,num):
    print(fd_list[i][0], fd_list[i][1])
    
print('--------------------------------------------------------\n')
    
print('bigrams:\n')
fd = big_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
for i in range(0,num):
    x, y= fd_list[i][0]
    term = x + ' '+ y
    print(term, fd_list[i][1])

print('--------------------------------------------------------\n')
print('trigrams:\n')

fd = trig_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
for i in range(0,num):
    x, y, z= fd_list[i][0]
    term = x + ' '+ y + ' ' + z
    print(term, fd_list[i][1])

unigrams:

people 66954
social 64162
security 59281
care 51287
health 41356
would 40135
insurance 39921
trump 39077
healthcare 38641
expansion 38198
like 35822
state 29422
cuts 28059
need 26424
want 26282
food 23709
work 23698
programs 23589
money 22908
states 21239
free 20572
know 19507
also 19277
take 19129
government 19113
poor 19012
public 18829
many 18774
stamps 18513
budget 18252
--------------------------------------------------------

bigrams:

social security 56353
food stamps 18234
health care 16445
health insurance 8658
private insurance 5960
cuts social 4551
take away 4155
work requirements 3638
middle class 3531
trump budget 3182
Medicare Medicaid 3103
mental health 3037
budget cuts 3009
pre-existing conditions 2964
insurance companies 2753
poor people 2727
public schools 2648
public option 2572
programs like 2491
social programs 2358
minimum wage 2324
Social Security 2272
illegal immigrants 2250
single payer 2234
affordable care 2144
medical care 2128
billion billion 2044

## Finding the interesting terms

In [23]:
term = 'social security'
pattern = rf'(^|[^a-zA-Z]){term}([^a-zA-Z]|$)' #rf is for using a variable inside
df_temp = df[df['text_remove_stopwords'].apply(lambda x: re.search(pattern,x)!=None)]
print('# of tweets:',len(df_temp))



# of tweets: 54964


In [22]:
term = 'social security'
pattern = rf'(^|[^a-zA-Z]){term}([^a-zA-Z]|$)' #rf is for using a variable inside
df_temp = df[df['text_remove_stopwords'].apply(lambda x: re.search(pattern,x)!=None)]
print('# of tweets:',len(df_temp))

It's a shame there's no tangible items like tanks or planes that can be used to demonstrate the costs of Social Security, Medicare, and Medicaid. 


---------------------------------------------------------------------------------
@HandicapperBill @ewarren 20 trillion in accumulated deficits;
100 trillion + in unfunded mandates for Social Security, Medicare and Medicaid.
Trillions more in unfunded pensions that the feds have guaranteed.
It’ll take just one failed bond auction...
So, if a private entity, bankrupt long ago. 


---------------------------------------------------------------------------------
@LadyRedWave @jimdwrench @larryelder Illegal immigrants with Illegal Documents are still Illegal. Only US Citizen's have Citizen's Rights. They Cannot Vote in US Elections.

Cannot Have Free:
1. Food Stamps
2. Social Security
3. Welfare
4. Healthcare
5. Medicare
6. Medicaid 
7. Housing
8. Cell Phones
9. Social Scv. 


-------------------------------------------------------------------

In [ ]:
num = 5
temp = df_temp.sample(n=num)
for i in range(0,num):
    print(temp['unprocessed_text'].iloc[i],'\n\n')
    #print(temp['text_remove_stopwords'].iloc[i],'\n\n')
    print('---------------------------------------------------------------------------------')